<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/%D0%94%D0%972.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашняя работа №2

# TF-IDF



У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

создадим массив данных 

In [48]:
texts = "\u0412 \u0440\u0430\u043C\u043A\u0430\u0445 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u044F \u0432\u0435\u0434\u0451\u0442\u0441\u044F \u043F\u043E\u0434\u0433\u043E\u0442\u043E\u0432\u043A\u0430 \u0441\u043F\u0435\u0446\u0438\u0430\u043B\u0438\u0441\u0442\u043E\u0432 \u0432 \u043E\u0431\u043B\u0430\u0441\u0442\u0438 \u043F\u0440\u043E\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F, \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u043A\u0438 \u0438 \u044D\u043A\u0441\u043F\u043B\u0443\u0430\u0442\u0430\u0446\u0438\u0438 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u043D\u043D\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C \u0443\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u044F \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0447\u0435\u0441\u043A\u0438\u043C \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u043E\u043C (\u0410\u0421\u0423\u0422\u041F) \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u043E\u0433\u043E \u043D\u0430\u0437\u043D\u0430\u0447\u0435\u043D\u0438\u044F. \u041E\u0431\u0443\u0447\u0430\u044E\u0449\u0438\u0435\u0441\u044F \u043F\u043E\u043B\u0443\u0447\u0430\u044E\u0442 \u0433\u043B\u0443\u0431\u043E\u043A\u0438\u0435 \u0437\u043D\u0430\u043D\u0438\u044F \u0432 \u043E\u0431\u043B\u0430\u0441\u0442\u0438 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u043E\u0432 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0445 \u043E\u0442\u0440\u0430\u0441\u043B\u0435\u0439; \u043F\u0440\u043E\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u0438 \u0438\u0437\u0433\u043E\u0442\u043E\u0432\u043B\u0435\u043D\u0438\u044F \u043C\u0430\u0448\u0438\u043D\u043E\u0441\u0442\u0440\u043E\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0445 \u0438\u0437\u0434\u0435\u043B\u0438\u0439; \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0447\u0435\u0441\u043A\u043E\u0433\u043E \u043E\u0431\u043E\u0440\u0443\u0434\u043E\u0432\u0430\u043D\u0438\u044F; \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u043A\u0438 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0434\u043B\u044F SCADA \u0438 PLC; \u043F\u0440\u043E\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u043E\u043D\u043D\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C \u0438 \u0431\u0430\u0437 \u0434\u0430\u043D\u043D\u044B\u0445 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0432\u0440\u0435\u043C\u0435\u043D\u0438; \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u044F \u0410\u0421\u0423\u0422\u041F \u043D\u0430 \u043E\u0441\u043D\u043E\u0432\u0435 \u043A\u043E\u043C\u043F\u044C\u044E\u0442\u0435\u0440\u043D\u044B\u0445 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u043B\u0435\u0440\u043D\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C, \u0441\u0435\u0440\u0432\u0435\u0440\u043E\u0432 \u0438 \u0441\u0435\u0442\u0435\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F; \u043A\u0438\u0431\u0435\u0440\u0444\u0438\u0437\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u0441\u0438\u0441\u0442\u0435\u043C \u0438 \u043E\u0431\u043B\u0430\u0447\u043D\u044B\u0445 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0439. \u041F\u043E\u0441\u043B\u0435 \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u0432\u044B\u043F\u0443\u0441\u043A\u043D\u0438\u043A\u0438 \u0441\u043C\u043E\u0433\u0443\u0442 \u0441\u043E\u0431\u0438\u0440\u0430\u0442\u044C \u0438 \u043F\u043E\u0434\u0432\u0435\u0440\u0433\u0430\u0442\u044C \u0430\u043D\u0430\u043B\u0438\u0437\u0443 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E, \u0434\u0430\u043D\u043D\u044B\u0435 \u0438 \u0441\u0432\u0435\u0434\u0435\u043D\u0438\u044F, \u043F\u0440\u0438\u043C\u0435\u043D\u044F\u0435\u043C\u044B\u0435 \u0432 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u0435 \u043F\u0440\u043E\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u043D\u043D\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C; \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u044F\u0442\u044C \u0446\u0435\u043B\u0435\u0432\u044B\u0435 \u0437\u0430\u0434\u0430\u0447\u0438 \u043F\u0440\u043E\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u0410\u0421\u0423\u0422\u041F, \u0432\u044B\u0441\u0442\u0440\u0430\u0438\u0432\u0430\u0442\u044C \u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u043D\u0443\u044E \u0432\u0437\u0430\u0438\u043C\u043E\u0441\u0432\u044F\u0437\u044C, \u0432\u044B\u0434\u0435\u043B\u044F\u0442\u044C \u043F\u0440\u0438\u043E\u0440\u0438\u0442\u0435\u0442\u043D\u044B\u0435 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u044F; \u0440\u0430\u0437\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u0442\u044C \u043F\u0440\u043E\u0435\u043A\u0442\u044B \u043F\u043E \u0441\u043E\u0437\u0434\u0430\u043D\u0438\u044E \u0410\u0421\u0423\u0422\u041F." #@param {type:"string"}
texts= str(texts)
print(texts)

В рамках направления ведётся подготовка специалистов в области проектирования, разработки и эксплуатации автоматизированных систем управления технологическим процессом (АСУТП) различного назначения. Обучающиеся получают глубокие знания в области технологических процессов различных отраслей; проектирования и изготовления машиностроительных изделий; технологического оборудования; разработки программного обеспечения для SCADA и PLC; проектирования информационных систем и баз данных реального времени; построения АСУТП на основе компьютерных контроллерных систем, серверов и сетевого обеспечения; киберфизических систем и облачных технологий. После обучения выпускники смогут собирать и подвергать анализу информацию, данные и сведения, применяемые в процессе проектирования автоматизированных систем; определять целевые задачи проектирования АСУТП, выстраивать структурную взаимосвязь, выделять приоритетные направления; разрабатывать проекты по созданию АСУТП.


удалим знаки переноса строк и сделаем текст в нижнем регистре

In [49]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')
  print(texts)

в рамках направления ведётся подготовка специалистов в области проектирования, разработки и эксплуатации автоматизированных систем управления технологическим процессом (асутп) различного назначения. обучающиеся получают глубокие знания в области технологических процессов различных отраслей; проектирования и изготовления машиностроительных изделий; технологического оборудования; разработки программного обеспечения для scada и plc; проектирования информационных систем и баз данных реального времени; построения асутп на основе компьютерных контроллерных систем, серверов и сетевого обеспечения; киберфизических систем и облачных технологий. после обучения выпускники смогут собирать и подвергать анализу информацию, данные и сведения, применяемые в процессе проектирования автоматизированных систем; определять целевые задачи проектирования асутп, выстраивать структурную взаимосвязь, выделять приоритетные направления; разрабатывать проекты по созданию асутп.


удалим цифры

In [50]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''
  print(texts)

в рамках направления ведётся подготовка специалистов в области проектирования, разработки и эксплуатации автоматизированных систем управления технологическим процессом (асутп) различного назначения. обучающиеся получают глубокие знания в области технологических процессов различных отраслей; проектирования и изготовления машиностроительных изделий; технологического оборудования; разработки программного обеспечения для scada и plc; проектирования информационных систем и баз данных реального времени; построения асутп на основе компьютерных контроллерных систем, серверов и сетевого обеспечения; киберфизических систем и облачных технологий. после обучения выпускники смогут собирать и подвергать анализу информацию, данные и сведения, применяемые в процессе проектирования автоматизированных систем; определять целевые задачи проектирования асутп, выстраивать структурную взаимосвязь, выделять приоритетные направления; разрабатывать проекты по созданию асутп.


удалим знаки пунктуации

In [51]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''
print(texts)

в рамках направления ведётся подготовка специалистов в области проектирования  разработки и эксплуатации автоматизированных систем управления технологическим процессом  асутп  различного назначения  обучающиеся получают глубокие знания в области технологических процессов различных отраслей  проектирования и изготовления машиностроительных изделий  технологического оборудования  разработки программного обеспечения для scada и plc  проектирования информационных систем и баз данных реального времени  построения асутп на основе компьютерных контроллерных систем  серверов и сетевого обеспечения  киберфизических систем и облачных технологий  после обучения выпускники смогут собирать и подвергать анализу информацию  данные и сведения  применяемые в процессе проектирования автоматизированных систем  определять целевые задачи проектирования асутп  выстраивать структурную взаимосвязь  выделять приоритетные направления  разрабатывать проекты по созданию асутп 


удалим стоп слова

In [52]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)
  

 рамках направления ведётся подготовка специалистов области проектирования разработки эксплуатации автоматизированных систем управления технологическим процессом асутп различного назначения обучающиеся получают глубокие знания области технологических процессов различных отраслей проектирования изготовления машиностроительных изделий технологического оборудования разработки программного обеспечения scada plc проектирования информационных систем баз данных реального времени построения асутп основе компьютерных контроллерных систем серверов сетевого обеспечения киберфизических систем облачных технологий обучения выпускники смогут собирать подвергать анализу информацию данные сведения применяемые процессе проектирования автоматизированных систем определять целевые задачи проектирования асутп выстраивать структурную взаимосвязь выделять приоритетные направления разрабатывать проекты созданию асутп


рассчитаем TF-IDF

In [53]:
records_count = 30
tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
wordCount = countVectorizer.fit_transform([texts])
TfIdf = tfIdfTransformer.fit_transform([texts])
names = countVectorizer.get_feature_names()

выведем результат на экран

In [54]:
import pandas as pd
df = pd.DataFrame(list(names), columns=['names'])
df = df.assign(Word_Count=wordCount.T.todense())
df = df.assign(TF_IDF=TfIdf.T.todense())
df = df.sort_values('TF_IDF', ascending=False)
print(df)

                                                names  Word_Count    TF_IDF
29                                             систем           5  0.474579
7                                      проектирования           5  0.474579
3                                               асутп           4  0.379663
2                           автоматизированных систем           2  0.189832
4                                         направления           2  0.189832
5                                         обеспечения           2  0.189832
6                                             области           2  0.189832
1                                  автоматизированных           2  0.189832
23                                         разработки           2  0.189832
24                            разработки программного           1  0.094916
21                         процессом асутп различного           1  0.094916
22                                         различного           1  0.094916
27          